In [1]:
#note that this should be run with the "openfst" kernel to allow us to open the correct python bindings for openfst,
# pywrapfst

In [7]:
import pickle
import Levenshtein
def load_pickle(filename):
    with open(filename, 'rb') as file:
        return(pickle.load(file))
import pandas as pd
import numpy as np
import sys
sys.path.append('/usr/local/lib/python3.6/site-packages')
from utils import wfst

In [8]:
all_tokens_phono = load_pickle('likelihood_inputs/all_tokens_phono.obj')
cmu_in_initial_vocab = load_pickle('likelihood_inputs/cmu_in_initial_vocab.obj')
initial_vocab = load_pickle('likelihood_inputs/initial_vocab.obj')
priors_for_age_interval = load_pickle('likelihood_inputs/priors_for_age_interval.obj')

# Levenshtein Distance

In [9]:
def get_edit_distance_matrix(all_tokens_phono, prior_data, initial_vocab,  cmu_2syl_inchildes):    
    '''
    Get an edit distance matrix for matrix-based computation of the posterior

    all_tokens_phono: corpus in tokenized from, with phonological transcriptions
    prior_data: priors of the form output by `compare_successes_failures_*`
    initial_vocab: word types corresponding to the softmask mask
    cmu_2syl_inchildes: cmu pronunctiations, must have 'word' and 'ipa_short' columns 
    '''

    bert_token_ids = prior_data['scores']['bert_token_id']
    ipa = pd.DataFrame({'bert_token_id':bert_token_ids}).merge(all_tokens_phono[['bert_token_id',
        'actual_phonology_no_dia']])

    iv = pd.DataFrame({'word':initial_vocab})
    iv = iv.merge(cmu_2syl_inchildes, how='left')

    levdists = np.vstack([np.array([Levenshtein.distance(target,x) for x in iv.ipa_short
    ]) for target in ipa.actual_phonology_no_dia])    
    return(levdists)    

In [10]:
levdists = get_edit_distance_matrix(all_tokens_phono, priors_for_age_interval, initial_vocab,  cmu_in_initial_vocab)

In [11]:
levdists.shape

(11718, 7904)

# WFST Distance

In [17]:
%pdb

Automatic pdb calling has been turned ON


In [27]:
import imp
imp.reload(wfst)

<module 'utils.wfst' from '/home/stephan/notebooks/nicole/child-directed-listening/utils/wfst.py'>

In [28]:
wfst_dists = wfst.get_wfst_distance_matrix(all_tokens_phono, priors_for_age_interval, initial_vocab,  cmu_in_initial_vocab,
    'fst/chi-1.txt', 'fst/chi_phones.sym')

/home/stephan/utils/openfst-1.8.1/openfst/lib/python3.6/site-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/home/stephan/notebooks/nicole/child-directed-listening/utils/wfst.py:152: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ats_end[0,2] = ''
/home/stephan/notebooks/nicole/child-directed-listening/utils/wfst.py:153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = 

In [29]:
wfst_dists

array([[2.91034475e-07, 9.70735108e-10, 3.67834322e-09, ...,
        7.37326126e-07, 8.21103669e-08, 4.29370811e-09],
       [1.52810475e-05, 1.07906226e-02, 3.85958957e-04, ...,
        3.55403403e-07, 7.24668517e-03, 5.87386341e-08],
       [1.54677852e-05, 1.29174011e-07, 1.94688503e-06, ...,
        5.18680997e-05, 2.73407839e-06, 6.45766567e-06],
       ...,
       [3.71623671e-06, 3.66785446e-07, 1.41435053e-05, ...,
        1.57430170e-08, 4.41272637e-07, 3.82429106e-06],
       [1.99342754e-05, 7.02976751e-07, 6.30810697e-07, ...,
        2.25177957e-06, 3.33650960e-07, 8.37396635e-08],
       [8.37781899e-01, 2.10227361e-03, 7.96600896e-03, ...,
        8.34289234e-04, 3.93670646e-03, 9.49116777e-05]])